<a href="https://colab.research.google.com/github/So-AI-love/ReinformentLearning-QuantumWellStatePreparation/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qutip

In [ ]:
import numpy as np
from numpy import pi
import pandas as pd
import random
import os
from qutip import *
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.optimizers import Adam
from collections import deque
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

/Users/jyj/anaconda3/envs/PythonData/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
#Solve the eigen-states and overlappings(see fig3 and fig4 in arXive:1812.04381) 
import numpy as np
from math import sin,pi 
import math
import scipy.optimize
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

N_Keep=6 #the number of energy levels we keep. Higher levels are neglected.
E0=pi**2/2 #gound state energy in natural unit system, used to rescale energy and alpha

def generate_bands_and_overlap(epsilon=0.01):
        a=1/2+epsilon
        b=1/2-epsilon
        
        def solve(alpha,solution):
            def F(k):
                c=2*alpha
                return k*np.sin(k*(a+b))+c*np.sin(k*a)*np.sin(k*b)
            solution=fsolve(F,solution) 
            return solution

       
        solution=np.array([s*pi for s in range(1,N_Keep+1)]) #without Dirac function
        Energies=[]
        kn=[]
        AS=[]
       
        alpha_list=np.array(range(10000))*0.2*E0 ##choose alpha list
        for alpha in alpha_list:
            solution=solve(alpha,solution)
            energy=[s**2/2/E0   for s in solution]  #wave vector to energy
            As=[(a/2-np.sin(2*k*a)/(4*k)+np.sin(k*a)**2/np.sin(k*b)**2*(b/2-np.sin(2*k*b)/(4*k)))**-1 for k in solution]
            Energies.append(energy)
            AS.append([np.sqrt(abs(s)) for s in As])
            kn.append(solution)
     
        Energies=np.matrix(Energies)
        kn=np.matrix(kn)
        AS=np.matrix(AS)
        alpha_list/=E0  
        #on the above, we rescale energy and alpha_list for plotting purpose, in the following, 
        #let's undo the rescale

        ###store the eig data####:
        Energies=Energies*E0
        alpha_list=alpha_list*E0 
        overlap=np.zeros((len(kn),N_Keep,N_Keep))
        for k in range(len(kn)):
            am=np.matrix(np.zeros((N_Keep,N_Keep)))
            for s1 in range(N_Keep):
                for s2 in range(N_Keep):
                    if s1!=s2:
                        dn=(Energies[k,s2]-Energies[k,s1])
                        up=AS[k,s2]*np.sin(kn[k,s2]*a) * AS[k,s1]*np.sin(kn[k,s1]*a)
                        ratio=up/dn
                        am[s1,s2]=ratio
            overlap[k,:,:]=am
      
        np.savetxt('./data/alpha_list_e_'+'{:.3f}'.format(epsilon)+'.txt', alpha_list, delimiter=',')
        np.savetxt('./data/Energies_e_'+'{:.3f}'.format(epsilon)+'.txt', Energies, delimiter=' ')
        np.savetxt('./data/overlap_e_'+'{:.3f}'.format(epsilon)+'.txt', overlap.flatten(), delimiter=' ')
        return alpha_list,Energies, overlap

N_asymmetries=6
for n in range (N_asymmetries):
    #     asymmetry = round(0.53 + 0.002*n,3)
    asymmetry=round(0.03+0.002*n,3)
    alpha_list,Energies, overlap=generate_bands_and_overlap(epsilon=asymmetry)

In [ ]:
# AS0,kn0,alpha_list0,Energies0, overlap0=generate_bands_and_overlap(epsilon=0.01), the fig3 and fig4 in arXive:1812.04381
# were reproduced exactly!  Helpful codes in SolveEig.ipynb, or, use the following lines for Fig3:
#alpha_list=alpha_list0/E0;alpha_list=alpha_list[:100]
# Energies=Energies0/E0;Energies=Energies[:100]
# kn=kn0[:100]
# AS=AS0[:100]
# %matplotlib inline
# _=plt.plot(alpha_list,  Energies[:,0], 'r--', alpha_list,  Energies[:,1], 'b-', 
#            alpha_list, Energies[:,2], 'g-',alpha_list,  Energies[:,3], 'y-')
# _=plt.ylabel('F function')
# plt.show()
# epsilon=0.01
# a=1/2+epsilon
# amp=[]
# for k in range(len(kn)):
#     am=[]
#     for s in range(1,6):
#         dn=(Energies[k,s]-Energies[k,0])*E0
#         up=AS[k,s]*np.sin(kn[k,s]*a) * AS[k,0]*np.sin(kn[k,0]*a)
#         ratio=up/dn
#         am.append(ratio)
#     amp.append(am)
# amp=np.matrix(amp)
# amp=abs(amp)
# _=plt.plot(alpha_list, amp[:,0],'b*-',alpha_list, amp[:,1], 'r--', alpha_list, amp[:,2], 'b-', 
#            alpha_list,amp[:,3], 'g*-',alpha_list, amp[:,4], 'y-')
# _=plt.ylabel('F function')
# plt.ylim(0, 0.01)
# plt.xlim(0, 20)
# plt.show()

In [ ]:
class Szilard_env:
    
    N_keep=6
    def __init__(self):

        self.max_lvl = self.N_keep
        
        self.viewer = None
        self.action_space = np.array([-1024,-512,-256,-128,-64,-32,-16,-8,-4,-2,0
                                      ,2,4,8,16,32,64,128,256,512,1024])*10
        self.action_space = np.reshape(self.action_space, (-1,1))
        
        self.state_space = np.array([0,0])
        self.state_space = np.reshape(self.state_space, (-1,1))

        self.max_alfa = 2000.0*E0       #set the maximun alfa 
        self.t_max = 5                 #time duration:[0,T], T=t_max
        self.N_alfa = 50               #set #alpha = #state= #time moments= #actions
        self.alfa_dt = self.t_max/self.N_alfa    #time interval
        
        self.sigma1 = 0.05    # punishment for finite probs of excited states with n>=3 
        self.sigma2 = 0.05    # punishment for different probs of the lowest two states
        
        self.best_protocol = [None]*self.N_alfa        
        self.protocol = [None]*self.N_alfa             
        self.best_reward = 0                        #best_reward和score的关系是啥 
        self.scores = []   #memorize reward for each episode when it is ended.
        
        self.interp_energies = []      
        self.interp_overlaps = []
        k = 0
        
        self.N_asymmetries = 6
        self.asymmetries = []
        
        self.N_t=1000; #for each episode, N_t is # of discret time we use in wave function evolution
        
        for n in range (self.N_asymmetries):
            asymmetry = round(0.53 + 0.002*n,3)
            self.asymmetries.append(asymmetry)                     #考虑哪些个不对称性的case
            
            epsilon=asymmetry-0.5
            energy, overlap = self.interp_generator(epsilon)     #d对每个不对称case, 生成能级和交叠作为时间的差值函数（generator形式）
            self.interp_energies.append(energy)      
            self.interp_overlaps.append(overlap)
        
        ## Here we can change the reward distribution to favour certain asymmetries
        
        #r_min  = 0.50
        #N_0 = -self.N_asymmetries/np.log(r_min)
        #x = np.array([n for n in range (self.N_asymmetries)])
        #self.reward_distribution = np.exp(-x/N_0)
        self.reward_distribution = np.ones(self.N_asymmetries)
            
    ## Interpolate data files to calculate eigenstates of the single-particle-box.  
    
    def interp_generator(self,epsilon):
        N_keep=self.N_keep
        #给定不对称位置，N_keep个能量和波函数叠加作为时间函数的数据，再生成差值函数
        
        tab_alfa = np.loadtxt('./data/alpha_list_e_'+'{:.3f}'.format(epsilon)+'.txt', delimiter = ',')        
        energies = np.loadtxt('./data/Energies_e_'+'{:.3f}'.format(epsilon)+'.txt', delimiter = ' ')
        overlap = np.loadtxt('./data/overlap_e_'+'{:.3f}'.format(epsilon)+'.txt', delimiter = ' ')
        overlap = np.reshape(overlap, [max_lvl,max_lvl,-1], order = 'F')
        overlap=np.transpose(overlap,(1,0,2))

        interp_energy = [None]*N_keep

        for n in range(N_keep):
            f = interp1d(tab_alfa,energies[:,n], kind = 'cubic')   #给定n‘th level,有一系列的能量对应一系列的alpha值（不同时刻）
            interp_energy[n] = f  #这里的f是一个函数“returns a function whose call method uses interpolation to find the value of new points.”

        interp_overlap = [[0] *N_keep for i in range(N_keep)] #给定n'th level,它和另外的每一个level之间都有overlap

        for n in range(N_keep):
            for m in range(N_keep):
                interp_overlap[n][m] = interp1d(tab_alfa,overlap[n,m,:], kind = 'cubic') #对应每两个level,对每一个时刻（或alpha的值）都有overlap的值

        return interp_energy, interp_overlap
    
    ## Interpolate the discrete protocol obtained from the agent at the end of each episode, and calculate the reward.
    def interp_reward(self, N_t):

        time_interp = np.array([ n*self.alfa_dt for n in range (self.N_alfa+1) ]) #线性时间
        
        alfa_interp = interp1d( time_interp,self.protocol, kind = 'cubic')  #protocol作为时间的函数进行差值
      
        time = np.linspace(0,self.t_max,N_t)       #这里N_t可以大大于N_alfa吧   
        alfa = alfa_interp(time)                     #
        alfa = np.clip(alfa, 0, self.max_alfa)   #限制alpha在[0,5000]之间, 对应函数interaction_int里面的alfa_in
    
        dt = time[1]-time[0]  
        max_lvl = self.max_lvl
 
        E = []
        for t in range(N_t):
            E.append([self.interp_energy[n](alfa[t]) for n in range(max_lvl)]) #self.interp_energy 是给定asymmetry的差值函数
        E = np.array(E)
        
        phi = []
        for t in range(N_t):
                integral = [np.trapz(x=time[0:t+1], y= E[0:t+1,n]) for n in range(max_lvl)]
                phi.append(integral)  #对应每个时刻，都可以求出相位表达式，即文2的appendix: A(4)
        phi = np.array(phi)   

        def interaction_int(n, alfa_in, f = []):
            test = sum([self.interp_overlap[n][m](abs(alfa_in))*np.exp(1j*(phi[t,n] - phi[t,m]) )*f[m] for m in range(max_lvl) if m != n ])*dalfa
            return test  #根据我的推导，这里的interp_overlap[n][m](abs(alfa_in))=<psi(n)|dH/dt|psi(m)>/(E_n-E_m)
        ##问题是，作为函数变量的alfa_in怎么能提供dH/dt里面的alpha对时间求导项？最后乘的dalfa？  这里需要乘一个

        C = np.array([ 0 for n in range(max_lvl) ])
        C[0] = 1
        C_new = np.array( [ 0 for n in range(max_lvl) ])
        C_t = []
        C_t.append(C)

        for t in range(N_t -1):
            
            dalfa = (alfa[t+1] - alfa[t])/dt   #alpha的导数

            k1 = np.array([dt*interaction_int(n, alfa[t], C ) for n in range(max_lvl)])
            k2 = np.array([dt*interaction_int(n, alfa[t]+ dalfa*dt/2, C + k1/2 ) for n in range(max_lvl)])
            k3 = np.array([dt*interaction_int(n, alfa[t]+ dalfa*dt/2, C + k2/2 ) for n in range(max_lvl)])
            k4 = np.array([dt*interaction_int(n, alfa[t]+ dalfa*dt, C + k3) for n in range(max_lvl)])

            C_new = C + (k1 + 2*k2 + 2*k3 + k4)/6   #这是某种迭代方法

            C = C_new

            C_t.append(C)#有必要把每个时刻的波函数都保存下来吗？

        c1 = abs(C[0])**2
        c2 = abs(C[1])**2
        
        reward = np.exp( -((c1 + c2 - 1))**2/self.sigma1 - ((c1 - c2))**2/self.sigma2 )#嗯，这两个要求都需要成立！
        

        #leakage = sum( abs(C[n])**2 for n in range(2,max_lvl) )
        
        #print("Normalization_int: {}" .format(sum(abs(C)**2)))
        
        return reward   

    ## Reset initial state after every episode     
    def reset(self):

        self.r = random.randrange(self.N_asymmetries)            #随便找一个偏离对称的几何结构，即epsilon,或者,a,b
        self.interp_energy = self.interp_energies[self.r]   #确定差值函数（generator）
        self.interp_overlap = self.interp_overlaps[self.r]
        
        self.time = 0.0
        self.alfa = 0.0
        self.steps = 0
        
        self.protocol = []               
        self.protocol.append(0)             #初始0时刻alpha为零
        self.action_sequence = []

        self.cum_reward = 0.0
                
        self.state =  [self.alfa/self.max_alfa] + [self.time/self.t_max]  #都是零，为何这样赋值？
        
        return np.array(self.state)       
    
    ## Take one step, and give reward as feedback.
    def step(self, action): 
        
        self.action_sequence.append(action)
        self.time += self.alfa_dt  #真实时间
        self.dalfa = self.action_space[action]  
                
        self.new_alfa = self.alfa + self.dalfa*self.alfa_dt  
        
        if self.new_alfa < 0.0:
            reward = -10
        elif self.new_alfa > self.max_alfa:
            reward = -10
        else:
            reward = 0

        self.new_alfa = np.clip(self.new_alfa, 0.0, self.max_alfa)
        
        self.steps += 1     
        self.protocol.append(self.new_alfa)
 
    
        self.alfa = self.new_alfa

        next_state = [self.alfa/self.max_alfa] + [self.time/self.t_max] 
        
        if self.steps == self.N_alfa:
            
#             N_t = 1000
            reward += 100*self.interp_reward(self.N_t)*self.reward_distribution[self.r]  #reward_distribution的元素都是1，为何要有它？          
            done = True            
            #if  reward > self.best_reward:
             #   self.best_protocol = self.protocol
             #   self.best_reward = reward
                #print('REWARD: {}'.format(reward))
        else:
            done = False
         
        
        self.cum_reward += reward  #reward只有两种方式可以得到值，区域越出或者最后self.steps == self.N_alfa
        
        if done == True:
            
            self.scores.append(self.cum_reward)
        
        return np.array(next_state), reward, done, {}

# Define agent

In [ ]:
# Based on implementation in Keras Deep Learning Cookbook by Rajdeep Dua et al. 

class Agent:
    
    def __init__(self, state_space, action_space):
        
        self.state_space = state_space
        self.action_space = action_space
        
        self.memory = deque(maxlen = 50000)  #用来存状态？
        self.gamma = 1.0
        self.epsilon = 1.0
        self.epsilon_decay = 0.99995          #这几个epsilon值是什么？ 
        self.epsilon_min = 0.05
        
        self.learning_rate = 0.001        
        
        self.model = self.build_model()
        self.target_model = self.build_model()
        
    def build_model(self):
        
        model = Sequential([
            Dense(24, input_dim = self.state_space.shape[0], activation = 'relu'), #这里的self.state_space应该是keras张量
            Dense(48, activation = 'relu'),
            Dense(24, activation = 'relu'),
            Dense(self.action_space.shape[0], activation = 'linear')
        ])
        
        model.compile(loss='mse', optimizer = Adam(lr = self.learning_rate))
        
        return model
    
    def get_action(self,state):
        
        if np.random.rand() <= self.epsilon:
            return random.randrange(len(self.action_space)-1) #-1 可以删除
        
        policy = self.model.predict(state)  
        best_action = np.argmax(policy[0])  #policy的shape是什么
        return best_action
 
    
    def add_to_memory(self, state, action, reward, next_state, done):
        
        self.memory.append((state, action, reward, next_state, done))
        

    def fit_from_memory(self, batch_size):
    
        batch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in batch:
            
            target = self.target_model.predict(state)
            
            if done:
                target[0][action] = reward
                
            else:
                Q_future = max(self.target_model.predict(next_state)[0])
                target[0][action] = reward + Q_future*self.gamma
                
            self.model.fit(state, target, epochs = 1, verbose = 0)   
        

                

# Set Parameters

In [ ]:
# env = Szilard_env()

In [ ]:
env = Szilard_env()
state_space = env.state_space
action_space = env.action_space

batch_size = 32
n_episodes = 40000
tau = 1e-3

agent = Agent(state_space, action_space)
agent.epsilon_decay = (0.05)**(1/n_episodes)
agent.memory = deque(maxlen = 200000)

score = [None]*n_episodes

output_dir = 'model_output/Model_6/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
print("Training on {} asymmetries in the range : {}".format(env.N_asymmetries,env.asymmetries))
print("Available actions: \n{}".format(env.action_space))
print("T_max = {}, Number of steps = {}, dt = {}".format(env.t_max, env.N_alfa, env.alfa_dt))
print("Hyperparameters: memorysize = {}, tau = {}, batch_size = {}".format(len(agent.memory),tau, batch_size))
print("Total number of experiences = {}".format(env.N_alfa*n_episodes))

Training on 6 asymmetries in the range : [0.53, 0.532, 0.534, 0.536, 0.538, 0.54]
Available actions: 
[[-10240]
 [ -5120]
 [ -2560]
 [ -1280]
 [  -640]
 [  -320]
 [  -160]
 [   -80]
 [   -40]
 [   -20]
 [     0]
 [    20]
 [    40]
 [    80]
 [   160]
 [   320]
 [   640]
 [  1280]
 [  2560]
 [  5120]
 [ 10240]]
T_max = 5, Number of steps = 50, dt = 0.1
Hyperparameters: memorysize = 0, tau = 0.001, batch_size = 32
Total number of experiences = 2000000


# Interact with environment

In [ ]:
done = False

for e in range(n_episodes):
    
    state = env.reset()
    state = np.reshape(state, (1,-1))
    
    for t in range(1000):
        
        action = agent.get_action(state)
        
        next_state, reward, done, _ = env.step(action)
                
        next_state = np.reshape(next_state, (1,-1))
        
        agent.add_to_memory(state, action, reward, next_state, done)
        
        state = next_state
        
        if done:
            print('episode: {}/{}, score: {:.3}, epsilon: {:.2}'.format(e,n_episodes, env.cum_reward, agent.epsilon))
            score[e] = env.cum_reward
            break
        
        if len(agent.memory) > batch_size:
            agent.fit_from_memory(batch_size)

            target_weights = np.array(agent.target_model.get_weights())
            current_weights = np.array(agent.model.get_weights())

            agent.target_model.set_weights(current_weights*tau + target_weights*(1-tau))
        
    if agent.epsilon > agent.epsilon_min:
        agent.epsilon *= agent.epsilon_decay
    
    if e % 100 == 0:
        
        agent.model.save_weights(output_dir + "weights_" + "{:04d}".format(e) + ".hdf5")
            

episode: 0/40000, score: -1e+02, epsilon: 1.0
episode: 1/40000, score: -2e+02, epsilon: 1.0
episode: 2/40000, score: -89.8, epsilon: 1.0
episode: 3/40000, score: -1.1e+02, epsilon: 1.0
episode: 4/40000, score: -79.9, epsilon: 1.0
episode: 5/40000, score: -70.0, epsilon: 1.0
episode: 6/40000, score: -1.1e+02, epsilon: 1.0
episode: 7/40000, score: -88.0, epsilon: 1.0
episode: 8/40000, score: -1.2e+02, epsilon: 1.0
episode: 9/40000, score: -49.9, epsilon: 1.0
episode: 10/40000, score: -12.9, epsilon: 1.0
episode: 11/40000, score: -2.1e+02, epsilon: 1.0
episode: 12/40000, score: -17.7, epsilon: 1.0
episode: 13/40000, score: -1.6e+02, epsilon: 1.0
episode: 14/40000, score: -60.0, epsilon: 1.0
episode: 15/40000, score: 69.9, epsilon: 1.0
episode: 16/40000, score: -1.4e+02, epsilon: 1.0
episode: 17/40000, score: -1.3e+02, epsilon: 1.0
episode: 18/40000, score: -2e+02, epsilon: 1.0
episode: 19/40000, score: -1.9e+02, epsilon: 1.0
episode: 20/40000, score: -80.0, epsilon: 1.0
episode: 21/40000,

KeyboardInterrupt: 

In [ ]:
a.my

'dog'

In [ ]:
a.h()

dog
